# Equitable Access to Potential Parks

This is an indicator for the 2023 base blueprint. 

We modified this slighty in 2023 year to add in zero values to make this indicator work better in our online viewers.

Created by Amy Keister, last run by Amy Keister on 21 June 2023. It took 33 minutes to run.

In [1]:
import os
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Equit\Equitab.gdb"

In [41]:
# define final outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Equit\EquitableAccessToPotentialParks.tif"

In [5]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [6]:
# define inputs
ParkRank= r"F:\GIS_DATA\DecisionSupportTools\TrustForPublicLand\ParkServe_fgd_DataShare_08062021\ParkServe_fgdb_DataShare_08062021.gdb\ParkServe_ParkPriorityAreas_08062021"
svi= r"F:\GIS_DATA\Cultural_Resources\SocialVulnerabilityIndex\SVI2018_US\SVI2018_US_tract.shp"
#zmask= r"E:\WORKING\BaseBlueprint\2022\4_SubregionsAndEdges\SubregionsAndEdgesLargeRoads.gdb\UrbResRoadND"
nlcd= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"
roads= r"F:\GIS_DATA\Transporation\OpenStreetMap\roads_20210311.shp"

### Start Analysis

In [7]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [8]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Equit\Equitab.gdb


### TPL ParkServe Data


In [9]:
# convert to raster using the "ParkRank" field
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster(ParkRank, "ParkRank", "PR", "CELL_CENTER", "NONE", 30)

In [10]:
# reclassify
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Reclassify("PR", "Value", "NODATA 0;0 0;1 1;2 2;3 3", "DATA"); out_raster.save("PRrclss")

### wrangle CDC Social Vulnerability Index
For this indicator in 2021, we are only going to score areas that have a higer score on the social vulnerability index

In [11]:
# make a copy of the SVI lines for edits
arcpy.management.CopyFeatures(svi, "svi", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Equit\\Equitab.gdb\\svi'>

In [12]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster(in_features="svi", field="RPL_Themes", out_raster="RPL_Themes", cell_size=Rextent)

In [13]:
# use conditional statement to make a binary raster. We are testing the cuttoff. This test is RPL_THEMES greater
# than .65
out_raster = arcpy.sa.Con("RPL_Themes", 1, 0, "Value > .65"); out_raster.save("RPL_Themes_GT0_65")

In [14]:
# limit TPL data to areas with greater than .65 in the RPL_Themes SVI scores 
out_raster = arcpy.sa.Times("PRrclss", "RPL_THEMES_GT0_65"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [20]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [21]:
# export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [43]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 3:
        return '3 = Very high priority for a new park that would create nearby equitable access'
    elif value == 2:
        return '2 = High priority for a new park that would create nearby equitable access'
    elif value == 1:
        return '1 = Moderate priority for a new park that would create nearby equitable access'
    elif value == 0:
        return '0 = Not identified as a priority for a new park that would create nearby equitable access (within urban areas)'
"""

In [44]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Equit\\EquitableAccessToPotentialParks.tif'>

In [45]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 3:
		return 112
	if Value == 2:
		return 172
	if Value == 1:
		return 223
	else:
		return 255
		
def Reclass2(Value):
	if Value == 3:
		return 12
	if Value == 2:
		return 102
	if Value == 1:
		return 184
	else:
		return 255
		
def Reclass3(Value):
	if Value == 3:
		return 242
	if Value == 2:
		return 237
	if Value == 1:
		return 230
	else:
		return 255
"""

In [46]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Equit\\EquitableAccessToPotentialParks.tif'>

In [22]:
end = time.time()
print(end - start)

1652.321837902069
